In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_ref_colors(xlsx, refcolor_sheet):
    print("\n Loading Reference Colors ...")
    color_ref = xlsx.parse(refcolor_sheet)
    color_array = color_ref["colorname0"].unique()
    print(color_array[:5])
    return color_array

In [3]:
def load_categories(xlsx, category_sheet):
    print("\n Loading Category data ...")
    df_cat = xlsx.parse(category_sheet)
    df_cat.dropna(inplace=True)
    df_cat["Category ID"] = df_cat["Category ID"].astype("int32")
    df_cat= df_cat[["Category ID","Category Name"]]
    print(df_cat.sample(5))
    return df_cat

In [4]:
def map_category(x):
    corrections = {
                    "Boots": "Boots & Booties",
                    "T-shirts":"T Shirts", 
                    "Sports Bras":"Bras",
                    "Denim": "Jeans"
                  }    
    x = x.split(":")[0].strip()
    mapped = corrections.get(x,0)
    if corrections.get(x,0) != 0:
        x = corrections.get(x,0)
    return x

def load_occasion_data(xlsx, occassion_sheet, catagory_df):

    print("\n Loading Occasion data ...")
    df_occassion = xlsx.parse(occassion_sheet, header=None, usecols=0)
    df_occassion.columns=["category"]
    
    """Remove lines with only category name and no occasion mapping and blank lines""" 
    df_occassion["category"] = df_occassion[df_occassion["category"].map(lambda x : ":" in x)]
    df_occassion.dropna(inplace=True)
    
    """Ensure category is one of the categories in Category-subcategory list""" 
    df_occassion["temp"] = df_occassion["category"].apply(lambda x : " ".join([i.strip() for i in x.split(":")[1].strip().split(",")] ))
    df_occassion["category"] = df_occassion["category"].apply(lambda x : map_category(x))
    df_occassion["category"] = df_occassion["category"]

    """Create one row per occasion"""
    df_occassion = df_occassion.join(
        df_occassion.temp.str.split(expand=True).stack().reset_index(drop=True, level=1)
        .rename("occassion")).drop("temp",axis=1)

    df_occassion=pd.merge(df_occassion, catagory_df, left_on="category", right_on="Category Name", how="left").\
        drop("Category Name", axis=1).rename(columns = {"Category ID": "category_id"})

    """Categories "Sweaters", "Sweatshirt" and Suits will be dropped as there are no matching Category ID's"""
    df_occassion.dropna(inplace=True)
    df_occassion["category_id"] = df_occassion["category_id"].astype("int32")
    print(df_occassion.sample(5))
    return df_occassion
    



In [5]:
def load_brands(xlsx,brands_sheet):
    print("\n Loading Brands data ...")
    df_topbrands = xlsx.parse(brands_sheet)
    df_topbrands = df_topbrands["brand_name"].append(df_topbrands["brand_name_synonym"]).map(lambda x : "".join(x.split()).lower()).drop_duplicates().sort_values().reset_index(drop=True)
    print(df_topbrands.sample(5))
    return df_topbrands

In [6]:
def load_influencers(xlsx,influencer_sheet,influencers):

    print("\n Loading Influencer data ...")

    """ List of influencers to look for as """
    df_influencers = xlsx.parse(influencer_sheet)
    df_influencers["user_id"] = df_influencers["user_id"].map(lambda x : x[-13:])
    df_influencers["influencers"] = df_influencers["style_who_inspiries"].map(lambda x : [1 if re.search(i,x) else 0 for i in influencers ])
    df_influencers[influencers] = pd.DataFrame(df_influencers["influencers"].values.tolist(), index = df_influencers.index)
    df_influencers.drop(["style_who_inspiries", "influencers"], axis =1, inplace=True)
    df_influencers.fillna(0,inplace=True)
    print(df_influencers.sample(5))
    return df_influencers
    

In [7]:
def load_user_data(xlsx, user_sheet, influencers):

    print("\n Loading User data ...")
    df_users = xlsx.parse(user_sheet)

    user_fields = [
        "user_id",
        "style_age_range",
        "style_age_range_group",
        "items_in_wishlist",
        "style_brands_selected",
        "style_size_preference_none",
        "style_size_preference_petite",
        "style_size_preference_extra_long",
        "style_size_preference_plus",
        "style_size_preference_maternity",
        "style_size_preference_skipped",
        "style_vibe",
        "has_stype_vibe",
        "style_who_inspiries_skipped",
        "style_looks_wanted_dates",
        "style_looks_wanted_everyday",
        "style_looks_wanted_formal",
        "style_looks_wanted_nights",
        "style_looks_wanted_other",
        "style_looks_wanted_summer",
        "style_looks_wanted_travel",
        'style_looks_wanted_winter',
        "style_looks_wanted_work",
        "style_looks_wanted_workouts",
        "style_looks_wanted_skipped",
        'style_most_important_active', 
        'style_most_important_any',
        'style_most_important_beach', 
        'style_most_important_dress',
        'style_most_important_bags', 
        'style_most_important_jeans',
        'style_most_important_jump', 
        'style_most_important_nothing',
        'style_most_important_outwear', 
        'style_most_important_pants',
        'style_most_important_shoes', 
        'style_most_important_tops',
        'style_most_important_skipped'
        ]

    user_fillna_zero_columns = [
        "style_size_preference_none",
        "style_size_preference_petite",
        "style_size_preference_extra_long",
        "style_size_preference_plus",
        "style_size_preference_maternity",
        "style_looks_wanted_dates",
        "style_looks_wanted_everyday",
        "style_looks_wanted_formal",
        "style_looks_wanted_nights",
        "style_looks_wanted_other",
        "style_looks_wanted_summer",
        "style_looks_wanted_travel",
        'style_looks_wanted_winter',
        "style_looks_wanted_work",
        "style_looks_wanted_workouts",
        'style_most_important_active', 
        'style_most_important_any',
        'style_most_important_beach', 
        'style_most_important_dress',
        'style_most_important_bags', 
        'style_most_important_jeans',
        'style_most_important_jump', 
        'style_most_important_nothing',
        'style_most_important_outwear', 
        'style_most_important_pants',
        'style_most_important_shoes', 
        'style_most_important_tops',
        'style_most_important_skipped'
    ]

    user_drop_columns = [
        "style_age_range",
        "style_brands_selected",
        "has_stype_vibe",
        "style_who_inspiries_skipped",
        "items_in_wishlist"
    ]

    user_int_conversion_columns = [
        "style_age_range_group",
        "style_size_preference_none",
        "style_size_preference_petite",
        "style_size_preference_extra_long",
        "style_size_preference_plus",
        "style_size_preference_maternity",
        "style_size_preference_skipped",
        "style_looks_wanted_dates",
        "style_looks_wanted_everyday",
        "style_looks_wanted_formal",
        "style_looks_wanted_nights",
        "style_looks_wanted_other",
        "style_looks_wanted_summer",
        "style_looks_wanted_travel",
        'style_looks_wanted_winter',
        "style_looks_wanted_work",
        "style_looks_wanted_workouts",
        "style_looks_wanted_skipped",
        'style_most_important_active', 
        'style_most_important_any',
        'style_most_important_beach', 
        'style_most_important_dress',
        'style_most_important_bags', 
        'style_most_important_jeans',
        'style_most_important_jump', 
        'style_most_important_nothing',
        'style_most_important_outwear', 
        'style_most_important_pants',
        'style_most_important_shoes', 
        'style_most_important_tops',
        'style_most_important_skipped'
    ]

    """
    placeholder to impute style looks wanted columns
    for now drop the columns
    """
    style_looks_columns = [
        "style_looks_wanted_dates",
        "style_looks_wanted_everyday",
        "style_looks_wanted_formal",
        "style_looks_wanted_nights",
        "style_looks_wanted_other",
        "style_looks_wanted_summer",
        "style_looks_wanted_travel",
        'style_looks_wanted_winter',
        "style_looks_wanted_work",
        "style_looks_wanted_workouts",
    ] 

    df_users = df_users[user_fields]
    df_users["user_id"] = df_users["user_id"].map(lambda x : x[-13:])
    df_users["style_age_range_group"] = df_users["style_age_range_group"].fillna(5)
    df_users["style_vibe"] = df_users["style_vibe"].fillna("None")

    df_users[user_fillna_zero_columns] = df_users[user_fillna_zero_columns].fillna(0)
    df_users[user_int_conversion_columns] = df_users[user_int_conversion_columns].astype("int64")

    """ Appending influencer data"""
    df_users = pd.merge(df_users,df_influencers,left_on="user_id",right_on="user_id", how="left")
    df_users[influencers] = df_users[influencers].fillna(0).astype("int64")

    """Duplicates are issue to be adressed later - revisit this"""
    #df_users.duplicated(df_users.columns[1:]).sum()

    df_users.drop(user_drop_columns + style_looks_columns + ["style_vibe"], axis=1, inplace=True)

    print(df_users.sample(5))
    return df_users


# EDA for Item Data

In [8]:
def load_item_data(xlsx, items_sheet, df_cat, df_brands, df_colors):
    print("\n Loading Item data ...")
    item_columns_tokeep = [
            'user_id', 'brand_id', 'user_provided_brand_name', 'parsed_brand_name',
            'store_id', 'user_provided_store_name','parsed_store_name','product_id', 
            'item_name_lower', 'product_category_id', 'paid_price',
            'list_price', 'sale_price',
            'order_total_amt', 'size', 'email_dt', 'color_parsed']

    """ usecols not working. Seems to be a bug in the library. To be checked further"""
    df_items = xlsx.parse(items_sheet)
    df_items = df_items[item_columns_tokeep]


    df_items["user_id"] = df_items["user_id"].map(lambda x : x[-13:])

    """Dropping rows with null product ID"""
    df_items = df_items[~df_items["product_id"].isnull()]
    df_items["product_id"] = df_items["product_id"].map(lambda x : x[-8:])


    df_items["product_category_id"].fillna(0, inplace=True)
    df_items["product_category_id"] = df_items["product_category_id"].astype("int64")


    """Adding "on sale" column indicate a purchase of item in sale """
    df_items["on_sale"] = df_items["sale_price"].map(lambda x : True if x > 0 else False)
    df_items["part_of_order"] = df_items.apply(lambda x : True if x["order_total_amt"] > x["paid_price"] else False, axis=1)

    """Get brand id with blank from other two brand columns when avaialble, else set to None"""
    df_items["brand_id"] = df_items["brand_id"].fillna(df_items["parsed_brand_name"])
    df_items["brand_id"] = df_items["brand_id"].fillna(df_items["user_provided_brand_name"])
    condition = pd.notnull(df_items["brand_id"])
    df_items.loc[condition,"brand_id"] = df_items.loc[condition,"brand_id"].map(lambda x : "".join(x.split()).lower())
    df_items["brand_id"].fillna("None", inplace=True)

    """Get store id with blank from other two brand columns when avaialble, else set to None"""
    df_items["store_id"] = df_items["store_id"].fillna(df_items["parsed_store_name"])
    df_items["store_id"] = df_items["store_id"].fillna(df_items["user_provided_store_name"])
    condition = pd.notnull(df_items["brand_id"])
    df_items.loc[condition,"store_id"] = df_items.loc[condition,"store_id"].map(lambda x : "".join(x.split()).lower())

    """Set store id ro Brand ID when null"""
    df_items["store_id"] = df_items["store_id"].fillna(df_items["brand_id"])

    """Set store ID to none if still not available"""
    df_items["store_id"].fillna("None", inplace=True)


    """Generate a new column that flags whether a store(brand) is a top rand """
    df_items["top_brand"] = df_items["store_id"].map(lambda x : True if x in df_brands.values.tolist() else False )


    """ Generate color column : pick value from color_parsed, if not available check if color is mentioned
    in Item description. In both the cases, match against master color list 
    """
    df_items["item_name_lower"] = df_items["item_name_lower"].fillna("None")
    df_items["item_name_lower"] = df_items["item_name_lower"].map(lambda x : str(x).lower())

    df_items["color_parsed"] = df_items["color_parsed"].fillna("None")
    df_items["color_parsed"] = df_items["color_parsed"].map(lambda x : str(x).lower())


    color_parse_lst = [set(str(e)) for e in df_items["color_parsed"].str.split()]
    df_items["color"] = [e&set(df_colors) for e in color_parse_lst]
    df_items["color"] = [list(e)[0] if len(list(e))!=0 else "" for e in df_items["color"]]


    color_ntlst = [set(e) for e in df_items["item_name_lower"].str.split()]
    df_items["color_tmp1"] = [e&set(df_colors) for e in color_ntlst]
    df_items["color_tmp1"] = [list(e)[0] if len(list(e))!=0 else "" for e in df_items["color_tmp1"]]

    df_items["color"] = df_items["color"].replace('', df_items["color_tmp1"])

    """Generate bins for price"""
    bins = np.array([0.0, 20.0, 50.0, 100.0, 200.0, 500.0, 1000.0, 20000.0])
    df_items["price_bin"] = np.digitize(df_items["paid_price"], bins)



    item_drop_columns = [
        "sale_price",
        "order_total_amt",
        "user_provided_brand_name",
        "parsed_brand_name",
        "user_provided_store_name",
        "parsed_store_name",
        "email_dt",
        "list_price",
        "color_tmp1",
        "color_parsed"
    ]
    df_items.drop(item_drop_columns, axis = 1,inplace=True)

    """Keep only if product elongs to master category list"""
    condition = df_items.product_category_id.map( lambda x : x in df_cat["Category ID"].values.tolist())
    df_items = df_items[condition]

    """ Drop duplicates. This is an issue, to e explored further on why there are duplicates"""
    df_items = df_items.drop_duplicates()

    """ Warning : Hard coding here. Eliminate Cosmetics and Kids clothes categories"""
    df_items = df_items[df_items["product_category_id"] <500]

    """Generate column combining category ID and Category Name """
    df_items = pd.merge(df_items,df_cat, left_on="product_category_id", right_on="Category ID", how="left",)
    df_items.product_category_id = df_items.apply(lambda x : str(x["product_category_id"]) + " - "+ str(x["Category Name"]), axis=1)

    df_items.drop("Category ID", axis =1, inplace=True)


    print(df_items.sample(5))
    return df_items


In [9]:
if __name__ == '__main__':
    excel_data = pd.ExcelFile('../../data/raw/mar19.xlsx')
    print("Sheet Names in the input XLSX:\n",excel_data.sheet_names)

    """Variables to avoid hardcoding of sheet names which might change later"""
    top_brands_sheet = "1. top_100_brands(brandname+syn"
    category_sheet = "9. category_ids"
    user_sheet = "14. 100_users_set"
    items_sheet = "15. 100_users_item_set"
    wishlist_sheet = "12. wishlist_items"
    influencer_sheet = "16. 100_user_influencer"
    refcolor_sheet = "2. reference_color"
    occassion_sheet = '6. category_and_subcategory'

    ''' hardcoded as influencers are mixed together as one long string in the data witout seperators'''
    influencers = [
        "ariellecharnas",
        "blaireadiebee",
        "blakevond",
        "chiaraferragni",
        "hannahbronfman",
        "jordynwoods",
        "manrepeller",
        "mayemusk",
        "nicolettemason",
        "seaofshoes",
        "somethingnavy",
        "weworewhat"
    ]

    """ Load and clean data """
    color_array = load_ref_colors(excel_data, refcolor_sheet)
    df_category = load_categories(excel_data, category_sheet)
    df_occassion = load_occasion_data(excel_data, occassion_sheet, df_category)
    df_topbrands = load_brands(excel_data, top_brands_sheet)
    df_influencers = load_influencers(excel_data,influencer_sheet, influencers)
    df_users = load_user_data(excel_data,user_sheet, influencers)
    df_items = load_item_data(excel_data, items_sheet, df_category, df_topbrands, color_array)

    ## Write dataframe to CSV file
    df_users.to_csv("../../data/processed/users.csv",index=False)
    df_items.to_csv("../../data/processed/items.csv",index=False)
    df_influencers.to_csv("../../data/processed/influencers.csv",index=False)
    df_category.to_csv("../../data/processed/categories.csv",index=False)
    df_occassion.to_csv("../../data/processed/occassions.csv",index=False)
    excel_data = None
    df_users = None
    df_items = None
    df_influencers = None
    df_category = None
    df_occassion = None



Sheet Names in the input XLSX:
 ['1. top_100_brands(brandname+syn', '2. reference_color', '3. personalization_rules(exampl', '4. sample_occasion', '5. influencer_color_rules', '6. category_and_subcategory', '7. user_subset', '8. item_subset', '9. category_ids', '10. brands_affinity', '11. styling_segments', '12. wishlist_items', '13. user_influencer', '14. 100_users_set', '15. 100_users_item_set', '16. 100_user_influencer']

 Loading Reference Colors ...
['white' 'aliceblue' 'antiquewhite' 'aquamarine' 'azure']

 Loading Category data ...
    Category ID Category Name
66          480       Wallets
62          440          Hats
70          520         Nails
28          155       Hoodies
53          320      Shoulder

 Loading Occasion data ...
    category occassion  category_id
214     Long     Night          132
47     Flats    Movies          220
139   Formal    Formal          144
88    Skirts    Dinner          121
53   Blouses      Life          111

 Loading Brands data ...
113  